# First Practical Work (Massive Computing)
#### Marina Gómez Rey (100472836) and Miguel Fernández Lara (100473125). Group 96
##### 20/10/2024


In [1]:
import MarinaMiguelFunctions as my
import time
import multiprocessing as mp
from multiprocessing.sharedctypes import Value, Array, RawArray
from multiprocessing import Process, Lock
import ctypes
import numpy as np
import time
import random

### Generating invertible matrix

In [3]:
def generate_invertible_matrix(size):
    while True:
        
        # We are using random.uniform but any other ranges of numbers could be used
        matrix = [[random.uniform(-1, 1) for _ in range(size)] for _ in range(size)]
        
        if np.linalg.det(matrix) != 0: # Allowed used for numpy for checking the determinant of the matrix
            return matrix

matrix_size = 100 # Change the number here to test multiple sizes

matrix = generate_invertible_matrix(matrix_size) # Generate a random invertible matrix

### Initializing the shared memory

In [5]:
data_buffer_size=len(matrix)**2 # The length of the data buffer must be n*n

print(data_buffer_size)  # Check the size of the data buffer, which will be stored into the shared memory

10000


In [7]:
# Parametrization of the number of cores

IS_HT=True #True if your computer have the HyperThreading feature activated, False if not.
NUMREPORTEDCORES=mp.cpu_count() 
# The number of real computational cores
if IS_HT:
    NUMCORES=(int) (NUMREPORTEDCORES/2)
else:
    NUMCORES=NUMREPORTEDCORES

print(NUMCORES)

8


In [9]:
shared_space= Array(ctypes.c_longlong, data_buffer_size) # Initialize the shared memory space with longlong c type (longest one we found)

shared_data=my.np.frombuffer(shared_space.get_obj(),dtype=np.float64) # Type float for large numbers

result_matrix = shared_data.reshape(len(matrix), len(matrix))  # In this matrix we will need to get the minors

print(result_matrix.shape) # Check the dimensions of the result matrix. It must be nxn

(100, 100)


In [11]:
%%time
my.determinant(matrix)  # Here we check how much the determinant function takes (LU factorization O(n^3))

CPU times: total: 78.1 ms
Wall time: 84.2 ms


3.5439920374013284e+52

### Parallelization per ROW using method of minors with LU determinant

In [13]:
%%time

n = len(matrix) # Retrieve the length of the matrix: n

row_indices = list(range(n)) # List of row indices to process

# Parallelize by rows
# The init arguments must be: shared space (to store the results in the shared memory), matrix shape (to reshape shared memory array) and 
# the current matrix (for it to become a global variable)
# We are using the multiprocessing library with the Pool method and map function.
with mp.Pool(processes=NUMCORES, initializer=my.init_sharedarray, initargs=[shared_space, (n, n), matrix]) as p:
    p.map(my.calculate_minor_for_row, row_indices)

# Now we have stored in the shared memory the whole matrix of minors

det_matrix = my.determinant(matrix) # Calculate the determinant of the whole matrix

invert_det = 1/det_matrix # Invert the value of the determinant

matrix_c = my.cofactor_matrix(result_matrix) # Get the cofactor matrix (multiply by 1 or -1)

matrix_t = my.transpose_matrix(matrix_c) # Transpose the matrix

inverse = my.multiply_number_by_matrix(invert_det,matrix_t) # Multiply the transposed matrix by the inverse of the determinant


CPU times: total: 156 ms
Wall time: 2min 53s


### Checking the solution (per row)

In [31]:
result = np.matmul(inverse, matrix) # We should get the identity

tolerance = 1e-7 # Some tolerance because they will not be exact 0's. 

identity = np.eye(n) # Numpy identity
    
print(np.allclose(result, identity, atol=tolerance)) # Print TRUE if the computation of the inverse is correct

# The lower the tolerance, the more probability of getting a False in the allclose method. It does not mean that the result is incorrect,
# it means that the significant figures taken were different and the round up was performed differently.

True


# Conclusion First Practical Work 

#### Introduction

The main goal of this first practical work is to understand how processes can be parallelized using the CPU and shared memory in our computers in order to make large complexity operations easily. In this case, we had to implement matrix inversion without using the numpy library. In order to perform the inversion, we chose to follow the method detailed in the practical work statement, since the computation of the minors is a highly parallelizable process and we wanted to reduce the time consumption in this process. 

The first step was to generate the invertible matrix, which could be done using numpy to check if the determinant is equal to 0. Then, we had to find a method to compute the matrix of minors. This is a high complexity process because for every cell given in the matrix there is a minor, which is computed finding a determinant of a matrix (n-1)x(n-1), which is the one resulting after removing the column and row of the target cell. Finally, we just had to perform the rest of sequential straightforward operations to find the inverse, which included finding the cofactor matrix (multiplying by 1 or -1), transposing this resulting matrix, and multiplying it by the inverse of the determinant of the initial matrix.


#### The parallelization process

Before starting the parallel processing, the shared memory had to be initilized. The shared space is an array of nxn positions, in which we will store "longlong" ctypes. It is the largest data type we could found in order to be able to store as many significant figures of the large determinant values computed from extremely big matrices. We have succesfully tested that having this ctype does not produce any overhead and therefore we decided to keep it. Besides, the shared data was initialized to store float64 numbers. This meant that the result of the determinats would be represented in floating point arithmetic. Finally, a result matrix was created by reshaping the shared space array, giving a more visual representation of the result while keeping the shared memory array format.

The multiprocessing library was used to implement the parallelization by using the "Pool" objects. We have decided to parallelize by rows as it has been the approach that led to the best results.

The "Pool" object created used the "map" method to parallelize, taking as arguments the indices of each row (a list with numbers from 0 to n-1) and as init arguments we passed the matrix (to create a global variable), the shared space and the matrix size.

#### Retreiving the minors

For every row input in the "map" method, we then computed sequentially the minors of each cell. Therefore, we needed to first find the most optimal method to retrieve the minor. This was achieved using vectorized operations instead of two nested for loops. The matrix resulting is the one that takes the indices of all rows unless the current one, and the same operation for the columns. This process is insignificant in terms of time consumption.


#### Finding the determinant

Finding the correct way to perform the determinant was a hustle, as we could not use numpy nor recursion. The lowest complexity algorithms we could think of had a complexity of O(n^3), which means three nested "for loops". After days of research, we could not find a more optimal way to do this. Therefore, we had to keep one of the cubic complexity method, which, by testing, we found the fastest was to decompose the matrix into LU factorization using gaussian elimination, which then provided the determinant of the matrix easily. However, this method is not highly scalable, since it takes a large amount of time for matrices over 200x200.


#### Storing in the shared memory

After the complete row of determinants is computed, it needs to be stored into the shared memory. To do this, we have implemented locks to provide a secure upload and retreival of data, avoiding overlapping of processes. After the lock is initialized, the complete row of determinants is stored inside the result matrix (reshaped shared space array).

#### Problems

The main problem arises with the overall wall time. Whilst the CPU time of the code is very low, the wall time is exponentially higher. Therefore, the conclusion we reached is that the overhead of the process is very high, which means that it is taking a huge amount of time to pass results to the shared memory and back, along with initializing the original matrix for each minor. However, we need to also take into consideration that the wall time also includes the queue from processes running inside our computer, which means that actually the timing to perform the matrix inversion is very low.
Regarding this problem, we decided to reduce by overhead by computing the parallelization by blocks instead of by rows, but that approach did not provide better results. Also, it is important to note that we divided by rows instead of by cells (fine granularity) to reduce this problem.

#### Other tried approaches

- LU decomposition: this technique consists on dividing the matrix into the multiplication of two. A lower triagular (L) and an upper triangular(U). Then, in order to compute the inverse the decomposition must be done and solving two triangular systems for each column of the identity matrix you obtain the inverse. We tried parallelizing this process by blocks or by columns. However, the results did not improve the previously explained method, so we remained with it.

- Gaussian elimination: this process involves converting the matrix into an upper triangular one through partial pivoting, scaling pivot rows, and eliminating entries below the pivot. Following this, the back substitution process is applied, which involves starting from the last row and eliminating values in the current column of all preceding rows while updating an identity matrix to construct the inverse. The process itself is extremely fast, making the inverse of a 100x100 matrix in less than two seconds. However, when parallelizing, the most expensive process is the eliminate the current column in other rows. We tried parallelizing this process keeping the rest sequential. However, that process is inside a loop and, overall it did not work as expected so we decided again to return to the original idea.

#### Timing Results

Using a MacBook Air 2020, with an Apple Silicon M1 processor (8 cores), the average times obtained were as follows:

- 10x10 matrix: 60 ms
- 50x50 matrix: 1.3 s
- 100x100 matrix: 57 s

We expect better performance on a machine with more cores and/or higher computational capacity.

#### Overall

As a final conclusion, we are satisfied with the performance of our implementation and we have succesfully learnt how to implement CPU parallelization in a fairly optimal way. We have also seen the importance of deciding which processes require to be parallelized and which ones can be sequential, because if everything is computed in parallel, issues with the locks can provide worse results than expected. The time results were not as expected in the original requirements, so we are very looking forward to seeing the solution for a further insight on our mistakes. 